In [ ]:
# For data generation
import sys
# !{sys.executable} -m pip install matplotlib --upgrade
import shutil
import os
import numpy as np
from glob import glob
import re
import joblib
import pandas as pd
import pickle
#random
from time import perf_counter

# sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score,root_mean_squared_error,mean_absolute_error,mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

#Plotting
import seaborn as sns
sns.set_style()
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.dpi'] = 200

import warnings
# warnings.filterwarnings("ignore", category=np.DeprecationWarning) 


from tqdm.auto import tqdm
# from tqdm.notebook import tqdm_notebook as tqdm

In [ ]:
datanums = sum([len(joblib.load(i)) for i in glob('cluster/*pickle')])
pairedcp=sns.color_palette('Paired')[8:10]

In [ ]:
def gen_lc(trainsize,testsize):
    with open(os.path.join(os.path.expanduser('~'),'DDCASPT2/drop.txt'),'r') as d:
        dropfeat = [i.replace('\n','') for i in d.readlines()]
        
    radius_range_dirs = []
    for i in glob('cluster/*.*'):
        try:
            radstr = float(os.path.basename(i))
            # if radstr>=1.4:
            radius_range_dirs.append(os.path.basename(i))
        except:
            continue
    
    radius_range = sorted(radius_range_dirs)
    
    train_ind,test_ind=radius_range[0::2],radius_range[1::2]
    
    train_ind = list(map(float,train_ind))
    test_ind = list(map(float,test_ind))
    train_ind, test_ind = train_test_split(train_ind+test_ind,train_size=trainsize, test_size=testsize, random_state=42)
    
    print(len(train_ind),len(test_ind))
    
    
    train = []
    test = []
    
    
    
    recover_train = []
    recover_test = []
    
    traincnt = 0
    testcnt = 0
    
    
    for k in train_ind:
        rad = f"{k:.2f}"
        traindf = pd.read_csv(os.path.join('cluster',rad,f'{rad}.csv'),compression='zip',index_col=0)
        # traindf = traindf[traindf['Pair_Energies'].abs()>=1e-7]
        train.append(traindf)
        recover_train.append((rad,traincnt,len(traindf)))
        traincnt+=1
            
    for l in test_ind:
        rad = f"{l:.2f}"
        testdf = pd.read_csv(os.path.join('cluster',rad,f'{rad}.csv'),compression='zip',index_col=0)
        # testdf = testdf[testdf['Pair_Energies'].abs()>=1e-7]
        test.append(testdf)                    
        recover_test.append((rad,testcnt,len(testdf)))
        testcnt+=1
    
# rename = {'h$_{qq}^{0}$':'h$_{q}$',
# '$(F_{q})_{0}$':'$F_{q}$',
# '$(F_{q}^{\\text{SCF}})_{0}$':'$F_{q}^{\\text{SCF}}$',
# '$(\\eta_{q})_{0}$':'$\\eta_{q}$',
# '$(\\omega_{q})_{0}$':'$\\omega_{q}$',
# '$(\\eta_{s})_{0}$':'$\\eta_{s}$',
# 'h$_{ss}^{0}$':'h$_{s}$',
# '$(F_{s}^{\\text{SCF}})_{0}$':'$F_{s}^{\\text{SCF}}$',
# '$(F_{s})_{0}$':'$F_{s}$',
# '$(\\omega_{s})_{0}$':'$\\omega_{s}$',
# '$(\\langle ss \\vert ss \\rangle)_{0}$':"$\\langle ss \\vert ss \\rangle$",
# '$(\\langle qq \\vert qq \\rangle)_{0}$':"$\langle qq \\vert qq \\rangle$",
# 'h$_{pp}^{0}$': '(h$_{p}$)$_{0}$',
# 'h$_{pp}^{1}$': '(h$_{p}$)$_{1}$',
# 'h$_{pp}^{2}$': '(h$_{p}$)$_{2}$',
# 'h$_{pp}^{3}$': '(h$_{p}$)$_{3}$',
# 'h$_{rr}^{0}$': '(h$_{r}$)$_{0}$',
# 'h$_{rr}^{1}$': '(h$_{r}$)$_{1}$',
# 'h$_{rr}^{2}$': '(h$_{r}$)$_{2}$',
# 'h$_{rr}^{3}$': '(h$_{r}$)$_{3}$',
# 'h$_{pq}^{0}$':'(h$_{pq}$)$_{0}$',
# 'h$_{pq}^{1}$':'(h$_{pq}$)$_{1}$',
# 'h$_{pq}^{2}$':'(h$_{pq}$)$_{2}$',
# 'h$_{pq}^{3}$':'(h$_{pq}$)$_{3}$',
# 'h$_{pr}^{0}$':'(h$_{pr}$)$_{0}$',
# 'h$_{pr}^{1}$':'(h$_{pr}$)$_{1}$',
# 'h$_{pr}^{2}$':'(h$_{pr}$)$_{2}$',
# 'h$_{pr}^{3}$':'(h$_{pr}$)$_{3}$',
# 'h$_{rs}^{0}$':'(h$_{rs}$)$_{0}$',
# 'h$_{rs}^{1}$':'(h$_{rs}$)$_{1}$',
# 'h$_{rs}^{2}$':'(h$_{rs}$)$_{2}$',
# 'h$_{rs}^{3}$':'(h$_{rs}$)$_{3}$',
# 'typ_0':'type_0',
# 'typ_1':'type_1',
# 'typ_2':'type_2',
# 'typ_3':'type_3',
# '1':"$\mathbf{b}$"         
# }


    rename = {'h$_{qq}^{0}$':'h$_{q}$',
    '$(F_{q})_{0}$':'$F_{q}$',
    '$(F_{q}^{\\text{SCF}})_{0}$':'$F_{q}^{\\text{SCF}}$',
    '$(\\eta_{q})_{0}$':'$\\eta_{q}$',
    '$(\\omega_{q})_{0}$':'$\\omega_{q}$',
    '$(\\eta_{s})_{0}$':'$\\eta_{s}$',
    'h$_{ss}^{0}$':'h$_{s}$',
    '$(F_{s}^{\\text{SCF}})_{0}$':'$F_{s}^{\\text{SCF}}$',
    '$(F_{s})_{0}$':'$F_{s}$',
    '$(\\omega_{s})_{0}$':'$\\omega_{s}$',
    '$(\\langle ss \\vert ss \\rangle)_{0}$':"$\\langle ss \\vert ss \\rangle$",
    '$(\\langle qq \\vert qq \\rangle)_{0}$':"$\langle qq \\vert qq \\rangle$",
    'h$_{pp}^{0}$': '(h$_{p}$)$_{0}$',
    'h$_{pp}^{1}$': '(h$_{p}$)$_{1}$',
    'h$_{pp}^{2}$': '(h$_{p}$)$_{2}$',
    'h$_{pp}^{3}$': '(h$_{p}$)$_{3}$',
    'h$_{rr}^{0}$': '(h$_{r}$)$_{0}$',
    'h$_{rr}^{1}$': '(h$_{r}$)$_{1}$',
    'h$_{rr}^{2}$': '(h$_{r}$)$_{2}$',
    'h$_{rr}^{3}$': '(h$_{r}$)$_{3}$',
    'h$_{pq}^{0}$':'(h$_{pq}$)$_{0}$',
    'h$_{pq}^{1}$':'(h$_{pq}$)$_{1}$',
    'h$_{pq}^{2}$':'(h$_{pq}$)$_{2}$',
    'h$_{pq}^{3}$':'(h$_{pq}$)$_{3}$',
    'h$_{pr}^{0}$':'(h$_{pr}$)$_{0}$',
    'h$_{pr}^{1}$':'(h$_{pr}$)$_{1}$',
    'h$_{pr}^{2}$':'(h$_{pr}$)$_{2}$',
    'h$_{pr}^{3}$':'(h$_{pr}$)$_{3}$',
    'h$_{rs}^{0}$':'(h$_{rs}$)$_{0}$',
    'h$_{rs}^{1}$':'(h$_{rs}$)$_{1}$',
    'h$_{rs}^{2}$':'(h$_{rs}$)$_{2}$',
    'h$_{rs}^{3}$':'(h$_{rs}$)$_{3}$',
    'typ_0':'$type_0$',
    'typ_1':'$type_1$',
    'typ_2':'$type_2$',
    'typ_3':'$type_3$',
    '1':"$\mathbf{b}$"}
        
    train_df = pd.concat(train).drop(columns=dropfeat).rename(columns=rename)
    test_df = pd.concat(test).drop(columns=dropfeat).rename(columns=rename)
    
    X_train = train_df.drop(columns=['Pair_Energies']).values
    X_test = test_df.drop(columns=['Pair_Energies']).values
    
    y_train = train_df['Pair_Energies'].values
    y_test = test_df['Pair_Energies'].values
    
    scaler=MinMaxScaler()
    X_train=scaler.fit_transform(X_train)
    X_test=scaler.transform(X_test)
    
    
    kwargs = dict(pd.read_excel("iron_VDZP_params.xlsx").values)
    model=XGBRegressor(**kwargs)
    model.fit(X_train,y_train)
    y_pred_train=model.predict(X_train)
    y_pred_test=model.predict(X_test)
    print(f"R2: {r2_score(y_train,y_pred_train):.4f},{r2_score(y_test,y_pred_test):.4f}")
    print(f"RMSE (mEh): {root_mean_squared_error(y_train,y_pred_train)*1e3:.4f},{root_mean_squared_error(y_test,y_pred_test)*1e3:.4f}")
    
    testcntrcvr = 0
    recover_test_list = []
    recover_test_E2 = []
    for j,k,l in recover_test:
        init_test = testcntrcvr
        testcntrcvr+=l
        recover_test_list.append((j,y_test[init_test:testcntrcvr],y_pred_test[init_test:testcntrcvr]))
        recover_test_E2.append((j,np.sum(y_test[init_test:testcntrcvr]),np.sum(y_pred_test[init_test:testcntrcvr])))
    
    
    traincntrcvr = 0
    recover_train_list = []
    recover_train_E2 = []
    for j,k,l in recover_train:
        init_train = traincntrcvr
        traincntrcvr+=l
        recover_train_list.append((j,y_train[init_train:traincntrcvr],y_pred_train[init_train:traincntrcvr]))
        recover_train_E2.append((j,np.sum(y_train[init_train:traincntrcvr]),np.sum(y_pred_train[init_train:traincntrcvr])))
    
    traincorrE2 = pd.DataFrame(recover_train_E2).rename(columns={0:'Radius ($\AA$)',1:'E$_{2}^{\text{Calculated}}$',2:'E$_{2}^{\text{Predicted}}$'}).set_index('Radius ($\AA$)')
    #.astype({"basis set":str,'Radius ($\AA$)':float,'E$_{2}^{\text{Calculated}}$':float,'E$_{2}^{\text{Predicted}}$':float})
    testcorrE2 = pd.DataFrame(recover_test_E2).rename(columns={0:'Radius ($\AA$)',1:'E$_{2}^{\text{Calculated}}$',2:'E$_{2}^{\text{Predicted}}$'}).set_index('Radius ($\AA$)')
    #.astype({"basis set":str,'Radius ($\AA$)':float,'E$_{2}^{\text{Calculated}}$':float,'E$_{2}^{\text{Predicted}}$':float})
    
    
    
    traincorrE2['CASPT2_E']=np.zeros(len(traincorrE2))
    traincorrE2['CASSCF_E']=np.zeros(len(traincorrE2))
    traincorrE2['E2']=np.zeros(len(traincorrE2))
    
    
    
    for i in traincorrE2.index:
        energy_df = pd.read_excel(f"cluster/{i}/{i}_energies.xlsx",index_col=0)
        traincorrE2.loc[i,'CASPT2_E']=energy_df.loc['CASPT2_E'].values
        traincorrE2.loc[i,'CASSCF_E']=energy_df.loc['CASSCF_E'].values
        # traincorrE2.loc[i,'E2']=energy_df.loc['E2'].values
    
    traincorrE2['E$_{\text{CASPT2}}^{\text{Predicted}}$'] = traincorrE2['CASSCF_E']+traincorrE2['E$_{2}^{\text{Predicted}}$']
    
    traincorrE2.rename(columns = {'CASPT2_E':'E$_{\text{CASPT2}}^{\text{Calculated}}$','CASSCF_E':'E$_{\text{CASSCF}}^{\text{Calculated}}$'},inplace=True)
    
    
    testcorrE2['CASPT2_E']=np.zeros(len(testcorrE2))
    testcorrE2['CASSCF_E']=np.zeros(len(testcorrE2))
    testcorrE2['E2']=np.zeros(len(testcorrE2))
    
    
    
    for i in testcorrE2.index:
        energy_df = pd.read_excel(f"cluster/{i}/{i}_energies.xlsx",index_col=0)
        testcorrE2.loc[i,'CASPT2_E']=energy_df.loc['CASPT2_E'].values
        testcorrE2.loc[i,'CASSCF_E']=energy_df.loc['CASSCF_E'].values
        # testcorrE2.loc[i,'E2']=energy_df.loc['E2'].values
    
    testcorrE2['E$_{\text{CASPT2}}^{\text{Predicted}}$'] = testcorrE2['CASSCF_E']+testcorrE2['E$_{2}^{\text{Predicted}}$']
    
    testcorrE2.rename(columns = {'CASPT2_E':'E$_{\text{CASPT2}}^{\text{Calculated}}$','CASSCF_E':'E$_{\text{CASSCF}}^{\text{Calculated}}$'},inplace=True)
    
    testcorrE2.reset_index(inplace=True)
    traincorrE2.reset_index(inplace=True)    
    
    return ((y_train,y_pred_train), (y_test,y_pred_test), (traincorrE2, testcorrE2))

In [ ]:

def plot(data,trainsize,save=False):
    pair_r2 = []
    pair_mae = []
    corr_r2 = []
    corr_mae = []
    for (i,((y_train,y_pred_train), (y_test,y_pred_test), (traincorrE2, testcorrE2))) in data:
        traincorrE2_true = traincorrE2['E$_{2}^{\text{Calculated}}$']
        traincorrE2_pred = traincorrE2['E$_{2}^{\text{Predicted}}$']
    
        testcorrE2_true = testcorrE2['E$_{2}^{\text{Calculated}}$']
        testcorrE2_pred = testcorrE2['E$_{2}^{\text{Predicted}}$']
        
        pair_r2.append((i,r2_score(y_train,y_pred_train),r2_score(y_test,y_pred_test)))
        pair_mae.append((i,1e3 * mean_absolute_error(y_train,y_pred_train),1e3 * mean_absolute_error(y_test,y_pred_test)))
    
        corr_r2.append((i,r2_score(traincorrE2_true,traincorrE2_pred),r2_score(testcorrE2_true,testcorrE2_pred)))
        corr_mae.append((i,1e3 * mean_absolute_error(traincorrE2_true,traincorrE2_pred),1e3 * mean_absolute_error(testcorrE2_true,testcorrE2_pred)))
    
    dfpair_r2 = pd.DataFrame(pair_r2,columns=['Train Size', "Train", "Test"])
    dfcorr_r2 = pd.DataFrame(corr_r2,columns=['Train Size', "Train", "Test"])
    dfpair_mae = pd.DataFrame(pair_mae,columns=['Train Size', "Train", "Test"])
    dfcorr_mae = pd.DataFrame(corr_mae,columns=['Train Size', "Train", "Test"])
    # dfcorr_r2.to_excel('corrr2.xlsx')
    print(dfcorr_r2)
    fig, ax = plt.subplots(2,2,sharex=True)
    sns.lineplot(dfpair_r2.melt(id_vars='Train Size',value_vars=['Train','Test']),x='Train Size',y='value', hue='variable',ax=ax[0][0],palette=pairedcp)
    sns.lineplot(dfcorr_r2.melt(id_vars='Train Size',value_vars=['Train','Test']),x='Train Size',y='value', hue='variable',ax=ax[0][1],markers='x',palette=pairedcp)
    ax[0][0].set_title(r'Pair-Energies ($\varepsilon_{qs}$)')
    ax[0][1].set_title(r'Correlation Energies (E$_{2}$)')    
    ax[0][0].set_xlim(0,700)
    ax[0][1].set_xlim(0,700)
    ax[0][0].set_ylim(-.74,1.1)
    ax[0][1].set_ylim(-.74,1.1)
    ax[0][0].set_ylabel("R$^{2}$")
    ax[0][1].set_ylabel("R$^{2}$")
    sns.lineplot(dfpair_mae.melt(id_vars='Train Size',value_vars=['Train','Test']),x='Train Size',y='value', hue='variable',ax=ax[1][0],palette=pairedcp)
    ax[1][0].set_ylim(0,0.25)
    sns.lineplot(dfcorr_mae.melt(id_vars='Train Size',value_vars=['Train','Test']),x='Train Size',y='value', hue='variable',ax=ax[1][1],palette=pairedcp)
    ax[1][1].set_ylim(0,8)
    ax[1][0].set_ylabel("MAE (mE$_{h}$)")
    ax[1][1].set_ylabel("MAE (mE$_{h}$)")
    plt.tight_layout()
    if save:
        plt.savefig('images/learning_curves.png',dpi=300,bbox_inches='tight')

In [ ]:
.1 * 500

In [ ]:
ranges=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8,0.9]

data = []
for i in ranges:
    # print(i,j)
    data.append((int(i * datanums),gen_lc(i,0.1)))
plt.show()

In [ ]:
plot(data,0.1,True)
plt.show()

In [ ]:

# ranges=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8,0.9]
# for j in ranges:
#     data = []
#     for i in ranges:
#         if j+i<=1:
#             # print(i,j)
#             data.append((int(i * datanums),gen_lc(i,j)))
#     plot(data,j)
#     plt.show()
    


In [ ]:
def gen_cut(cutoff):
    with open(os.path.join(os.path.expanduser('~'),'DDCASPT2/drop.txt'),'r') as d:
        dropfeat = [i.replace('\n','') for i in d.readlines()]
        
    radius_range_dirs = []
    for i in glob('cluster/*.*'):
        try:
            radstr = float(os.path.basename(i))
            # if radstr>=1.4:
            radius_range_dirs.append(os.path.basename(i))
        except:
            continue
    
    radius_range = sorted(radius_range_dirs)
    
    train_ind,test_ind=radius_range[0::2],radius_range[1::2]
    
    train_ind = list(map(float,train_ind))
    test_ind = list(map(float,test_ind))
    train_ind, test_ind = train_test_split(train_ind+test_ind, test_size=0.1, random_state=42)
    
    print(len(train_ind),len(test_ind))
    
    
    train = []
    test = []
    
    
    
    recover_train = []
    recover_test = []
    
    traincnt = 0
    testcnt = 0
    
    
    for k in train_ind:
        rad = f"{k:.2f}"
        traindf = pd.read_csv(os.path.join('cluster',rad,f'{rad}.csv'),compression='zip',index_col=0)
        traindf = traindf[traindf['Pair_Energies'].abs()>=cutoff]
        train.append(traindf)
        recover_train.append((rad,traincnt,len(traindf)))
        traincnt+=1
            
    for l in test_ind:
        rad = f"{l:.2f}"
        testdf = pd.read_csv(os.path.join('cluster',rad,f'{rad}.csv'),compression='zip',index_col=0)
        testdf = testdf[testdf['Pair_Energies'].abs()>=cutoff]
        test.append(testdf)                    
        recover_test.append((rad,testcnt,len(testdf)))
        testcnt+=1
    
    rename = {'h$_{qq}^{0}$':'h$_{q}$',
    '$(F_{q})_{0}$':'$F_{q}$',
    '$(F_{q}^{\\text{SCF}})_{0}$':'$F_{q}^{\\text{SCF}}$',
    '$(\\eta_{q})_{0}$':'$\\eta_{q}$',
    '$(\\omega_{q})_{0}$':'$\\omega_{q}$',
    '$(\\eta_{s})_{0}$':'$\\eta_{s}$',
    'h$_{ss}^{0}$':'h$_{s}$',
    '$(F_{s}^{\\text{SCF}})_{0}$':'$F_{s}^{\\text{SCF}}$',
    '$(F_{s})_{0}$':'$F_{s}$',
    '$(\\omega_{s})_{0}$':'$\\omega_{s}$',
    '$(\\langle ss \\vert ss \\rangle)_{0}$':"$\\langle ss \\vert ss \\rangle$",
    '$(\\langle qq \\vert qq \\rangle)_{0}$':"$\langle qq \\vert qq \\rangle$",
    'h$_{pp}^{0}$': 'h$_{p}^{0}$',
    'h$_{pp}^{1}$': 'h$_{p}^{1}$',
    'h$_{pp}^{2}$': 'h$_{p}^{2}$',
    'h$_{pp}^{3}$': 'h$_{p}^{3}$',
    'h$_{rr}^{0}$': 'h$_{r}^{0}$',
    'h$_{rr}^{1}$': 'h$_{r}^{1}$',
    'h$_{rr}^{2}$': 'h$_{r}^{2}$',
    'h$_{rr}^{3}$': 'h$_{r}^{3}$'}
    
    train_df = pd.concat(train).drop(columns=dropfeat).rename(columns=rename)
    test_df = pd.concat(test).drop(columns=dropfeat).rename(columns=rename)
    
    X_train = train_df.drop(columns=['Pair_Energies']).values
    X_test = test_df.drop(columns=['Pair_Energies']).values
    
    y_train = train_df['Pair_Energies'].values
    y_test = test_df['Pair_Energies'].values
    
    scaler=MinMaxScaler()
    X_train=scaler.fit_transform(X_train)
    X_test=scaler.transform(X_test)
    
    
    kwargs = dict(pd.read_excel("iron_VDZP_params.xlsx").values)
    model=XGBRegressor(**kwargs)
    model.fit(X_train,y_train)
    y_pred_train=model.predict(X_train)
    y_pred_test=model.predict(X_test)
    print(f"R2: {r2_score(y_train,y_pred_train):.4f},{r2_score(y_test,y_pred_test):.4f}")
    print(f"RMSE (mEh): {root_mean_squared_error(y_train,y_pred_train)*1e3:.4f},{root_mean_squared_error(y_test,y_pred_test)*1e3:.4f}")
    
    testcntrcvr = 0
    recover_test_list = []
    recover_test_E2 = []
    for j,k,l in recover_test:
        init_test = testcntrcvr
        testcntrcvr+=l
        recover_test_list.append((j,y_test[init_test:testcntrcvr],y_pred_test[init_test:testcntrcvr]))
        recover_test_E2.append((j,np.sum(y_test[init_test:testcntrcvr]),np.sum(y_pred_test[init_test:testcntrcvr])))
    
    
    traincntrcvr = 0
    recover_train_list = []
    recover_train_E2 = []
    for j,k,l in recover_train:
        init_train = traincntrcvr
        traincntrcvr+=l
        recover_train_list.append((j,y_train[init_train:traincntrcvr],y_pred_train[init_train:traincntrcvr]))
        recover_train_E2.append((j,np.sum(y_train[init_train:traincntrcvr]),np.sum(y_pred_train[init_train:traincntrcvr])))
    
    traincorrE2 = pd.DataFrame(recover_train_E2).rename(columns={0:'Radius ($\AA$)',1:'E$_{2}^{\text{Calculated}}$',2:'E$_{2}^{\text{Predicted}}$'}).set_index('Radius ($\AA$)')
    #.astype({"basis set":str,'Radius ($\AA$)':float,'E$_{2}^{\text{Calculated}}$':float,'E$_{2}^{\text{Predicted}}$':float})
    testcorrE2 = pd.DataFrame(recover_test_E2).rename(columns={0:'Radius ($\AA$)',1:'E$_{2}^{\text{Calculated}}$',2:'E$_{2}^{\text{Predicted}}$'}).set_index('Radius ($\AA$)')
    #.astype({"basis set":str,'Radius ($\AA$)':float,'E$_{2}^{\text{Calculated}}$':float,'E$_{2}^{\text{Predicted}}$':float})
    
    
    
    traincorrE2['CASPT2_E']=np.zeros(len(traincorrE2))
    traincorrE2['CASSCF_E']=np.zeros(len(traincorrE2))
    traincorrE2['E2']=np.zeros(len(traincorrE2))
    
    
    
    for i in traincorrE2.index:
        energy_df = pd.read_excel(f"cluster/{i}/{i}_energies.xlsx",index_col=0)
        traincorrE2.loc[i,'CASPT2_E']=energy_df.loc['CASPT2_E'].values
        traincorrE2.loc[i,'CASSCF_E']=energy_df.loc['CASSCF_E'].values
        # traincorrE2.loc[i,'E2']=energy_df.loc['E2'].values
    
    traincorrE2['E$_{\text{CASPT2}}^{\text{Predicted}}$'] = traincorrE2['CASSCF_E']+traincorrE2['E$_{2}^{\text{Predicted}}$']
    
    traincorrE2.rename(columns = {'CASPT2_E':'E$_{\text{CASPT2}}^{\text{Calculated}}$','CASSCF_E':'E$_{\text{CASSCF}}^{\text{Calculated}}$'},inplace=True)
    
    
    testcorrE2['CASPT2_E']=np.zeros(len(testcorrE2))
    testcorrE2['CASSCF_E']=np.zeros(len(testcorrE2))
    testcorrE2['E2']=np.zeros(len(testcorrE2))
    
    
    
    for i in testcorrE2.index:
        energy_df = pd.read_excel(f"cluster/{i}/{i}_energies.xlsx",index_col=0)
        testcorrE2.loc[i,'CASPT2_E']=energy_df.loc['CASPT2_E'].values
        testcorrE2.loc[i,'CASSCF_E']=energy_df.loc['CASSCF_E'].values
        # testcorrE2.loc[i,'E2']=energy_df.loc['E2'].values
    
    testcorrE2['E$_{\text{CASPT2}}^{\text{Predicted}}$'] = testcorrE2['CASSCF_E']+testcorrE2['E$_{2}^{\text{Predicted}}$']
    
    testcorrE2.rename(columns = {'CASPT2_E':'E$_{\text{CASPT2}}^{\text{Calculated}}$','CASSCF_E':'E$_{\text{CASSCF}}^{\text{Calculated}}$'},inplace=True)
    
    testcorrE2.reset_index(inplace=True)
    traincorrE2.reset_index(inplace=True)    
    
    return ((y_train,y_pred_train), (y_test,y_pred_test), (traincorrE2, testcorrE2))

In [ ]:
14-3

In [ ]:
data = []
for i in np.logspace(-14,-3,12):
    print(i)
    data.append((i,gen_cut(i)))


pair_r2 = []
pair_mae = []
corr_r2 = []
corr_mae = []
for (i,((y_train,y_pred_train), (y_test,y_pred_test), (traincorrE2, testcorrE2))) in data:
    traincorrE2_true = traincorrE2['E$_{2}^{\text{Calculated}}$']
    traincorrE2_pred = traincorrE2['E$_{2}^{\text{Predicted}}$']

    testcorrE2_true = testcorrE2['E$_{2}^{\text{Calculated}}$']
    testcorrE2_pred = testcorrE2['E$_{2}^{\text{Predicted}}$']
    
    pair_r2.append((i,r2_score(y_train,y_pred_train),r2_score(y_test,y_pred_test)))
    pair_mae.append((i,1e3 * mean_absolute_error(y_train,y_pred_train),1e3 * mean_absolute_error(y_test,y_pred_test)))

    corr_r2.append((i,r2_score(traincorrE2_true,traincorrE2_pred),r2_score(testcorrE2_true,testcorrE2_pred)))
    corr_mae.append((i,1e3 * mean_absolute_error(traincorrE2_true,traincorrE2_pred),1e3 * mean_absolute_error(testcorrE2_true,testcorrE2_pred)))

dfpair_r2 = pd.DataFrame(pair_r2,columns=['Cutoff', "Train", "Test"])
dfcorr_r2 = pd.DataFrame(corr_r2,columns=['Cutoff', "Train", "Test"])
dfpair_mae = pd.DataFrame(pair_mae,columns=['Cutoff', "Train", "Test"])
dfcorr_mae = pd.DataFrame(corr_mae,columns=['Cutoff', "Train", "Test"])




In [ ]:
dfpair_r2

In [ ]:
pairedcp=sns.color_palette('Paired')

In [ ]:
fig, ax = plt.subplots(2,2,sharex=True)
sns.lineplot(dfpair_r2.melt(id_vars='Cutoff',value_vars=['Train','Test']),x='Cutoff',y='value', hue='variable',ax=ax[0][0],palette=pairedcp[8:10])
sns.lineplot(dfcorr_r2.melt(id_vars='Cutoff',value_vars=['Train','Test']),x='Cutoff',y='value', hue='variable',ax=ax[0][1],markers='x',palette=pairedcp[8:10])
# ax[0][0].set_xlim(0,700)
# ax[0][1].set_xlim(0,700)
ax[0][0].set_title(r'Pair-Energies ($\varepsilon_{qs}$)')
ax[0][1].set_title(r'Correlation Energies (E$_{2}$)')
ax[0][0].set_xscale("log")
ax[0][1].set_xscale("log")
ax[0][0].set_ylim(-.74,1.1)
ax[0][1].set_ylim(-.74,1.1)
ax[0][0].set_xlim(10**-15,0)
# ax[0][1].set_xlim(-.74,1.1)
ax[0][0].set_ylabel("R$^{2}$")
ax[0][1].set_ylabel("R$^{2}$")
sns.lineplot(dfpair_mae.melt(id_vars='Cutoff',value_vars=['Train','Test']),x='Cutoff',y='value', hue='variable',ax=ax[1][0],palette=pairedcp[8:10])
ax[1][0].set_ylim(0,0.25)
sns.lineplot(dfcorr_mae.melt(id_vars='Cutoff',value_vars=['Train','Test']),x='Cutoff',y='value', hue='variable',ax=ax[1][1],palette=pairedcp[8:10])
ax[1][1].set_ylim(0,8)
ax[1][0].set_ylabel("MAE (mE$_{h}$)")
ax[1][1].set_ylabel("MAE (mE$_{h}$)")
plt.tight_layout()
plt.savefig('images/cutoff_curves.png',dpi=300,bbox_inches='tight')

In [ ]:
dfcorr_r2.sort_values(by=['Train','Test'])